In [ ]:
#작성자 : teadone
#작성일 : 20220308
#AOT Groundtruth.csv를 입력받아, COCO 데이터셋의 형태를 지닌 Groundtruth.json을 생성하는 코드
#Code that receives AOT Ground Truth.csv and generates Ground Truth.json in the form of COCO dataset

#Goundtruth.csv를 입력받아, id컬럼(탐지 객체 클래스)의 숫자를 제거 (ex Airborne11 -> Airborne)
#이후 탐지 클래스를 정리 후, COCO 데이터셋의 형태로 가공#Take Goundtruth.csv as input and remove the number of id column (detection object class) (ex Airborne11 -> Airborne)
#After cleaning up the detection class, it is processed in the form of COCO dataset.

In [1]:
import numpy as np
import json
import pandas as pd
import re

path = 'part2/ImageSets/groundtruth.csv'#input ground truth file path
save_json_path = 'part2/ImageSets/groundtruth_coco.json' # input ground truth fil path you want output it

data = pd.read_csv(path)

data['id'] = data['id'].fillna('None') # replace from Nan value to 'None' string
data['id'] = data['id'].str.replace(pat='[0-9]', repl=r'', regex=True) # remove number use regex pattern


images = []
categories = []
annotations = []

category = {}
category["supercategory"] = 'none'
category["id"] = 0
category["name"] = 'None'
categories.append(category)

data['fileid'] = data['img_name'].astype('category').cat.codes
data['categoryid']= pd.Categorical(data['id'],ordered= True).codes
data['categoryid'] = data['categoryid']+1
data['annid'] = data.index

def image(row):
    image = {}
    image["height"] = row.size_height
    image["width"] = row.size_width
    image["id"] = row.fileid
    image["file_name"] = row.img_name
    return image

def category(row):
    category = {}
    category["supercategory"] = 'None'
    category["id"] = row.categoryid
    print(row[6])
    category["name"] = row[6]
    return category

def annotation(row):
    annotation = {}
    area = (row.gt_right -row.gt_left)*(row.gt_bottom - row.gt_top)
    annotation["segmentation"] = []
    annotation["iscrowd"] = 0
    annotation["area"] = area
    annotation["image_id"] = row.fileid

    annotation["bbox"] = [row.gt_left, row.gt_top, row.gt_right -row.gt_left,row.gt_bottom-row.gt_top ]

    annotation["category_id"] = row.categoryid
    annotation["id"] = row.annid
    return annotation

for row in data.itertuples():
    annotations.append(annotation(row))

imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
for row in imagedf.itertuples():
    images.append(image(row))

catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
for row in catdf.itertuples():
    categories.append(category(row))

data_coco = {}
data_coco["images"] = images
data_coco["categories"] = categories
data_coco["annotations"] = annotations
json.dump(data_coco, open(save_json_path, "w"), indent=4)

Airborne
Airplane
Bird
Drone
Flock
Helicopter
None
